In [105]:
import requests
from bs4 import BeautifulSoup

base_url = "https://pandas.pydata.org/docs/reference/index.html"
response = requests.get(base_url)

In [106]:
soup = BeautifulSoup(response.text, "lxml")
navbar = soup.find_all(attrs={"class": "reference internal"})

In [107]:
ref_url = "https://pandas.pydata.org/docs/reference/"
ref_url+ navbar[0]['href']

'https://pandas.pydata.org/docs/reference/io.html'

In [108]:
first_level = {}
for nbar in navbar:
    text_val = nbar.text
    href = nbar['href']
    full_url = ref_url+ href
    if  "#" not in href:
        first_level.update({href:{"functions":[],"name":text_val,"url":full_url}})
    else:
        continue

In [109]:
for nbar in navbar:
    text_val = nbar.text
    href = nbar['href']
    full_url = ref_url+ href
    if "#" in href:
        parent_html = href.split("#")[0]
        if parent_html in first_level:
            first_level[parent_html]['functions'].append({"name":text_val,"url":full_url})
        else:
            print(text_val)

In [110]:
# first_level['io.html']['functions']
# https://pandas.pydata.org/docs/reference/io.html

## EXTRACTING FUNCTIONS

In [7]:
# parent_url = first_level['io.html']['url']

# parent_soup = BeautifulSoup(requests.get(parent_url).text,'lxml')
# url_id = "pickling"
# s1 = parent_soup.find(attrs={"id":url_id})
# s1

In [122]:
def get_links(id_elem,base_func_url,class_name):
    curr_urls = []
    try:
        func_urls = id_elem.find_all(attrs={"class":class_name})
        for odd_url in func_urls:
            func_url = odd_url.find('a')['href']
            curr_urls.append(base_func_url+func_url)
        return curr_urls
    except Exception as e:
        print(e)
        func_urls = id_elem.find(attrs={"class":class_name}).find('a')['href']
        curr_urls.append(base_func_url+func_url)
        return curr_urls
    finally:
        return curr_urls
    
base_func_url = "https://pandas.pydata.org/docs/reference/"
for first_level_name in first_level:
    parent_url = first_level[first_level_name]['url']
    parent_soup = BeautifulSoup(requests.get(parent_url).text,'lxml')
    all_urls = []
    for func in first_level[first_level_name]['functions']:
        url_id = func['url'].split("/")[-1].split("#")[-1]
        id_elem = parent_soup.find(attrs={"id":url_id})
        odd_urls = get_links(id_elem,base_func_url,"row-odd")     
        even_urls = get_links(id_elem,base_func_url,"row-even")     
        all_urls.extend(odd_urls)
        all_urls.extend(even_urls)
    first_level[first_level_name]['function_urls'] = all_urls
    # break

'NoneType' object is not subscriptable
'NoneType' object is not subscriptable


In [133]:
# from tqdm import tqdm
# def get_param_data(first_level):
#     for parent, values in first_level.items():
#         first_level[parent].update({"function_definitions":[]})
#         for func_url in tqdm(values['function_urls']):
#             func_response = requests.get(func_url)
#             func_soup = BeautifulSoup(func_response.content, "lxml",from_encoding="utf-8")
            
#             func_name = func_soup.find("h1").text.replace("#","") #remove #
#             try:
#                 elem = func_soup.find(attrs={"class":"sig sig-object py"})
#                 full_function = elem.text.replace("[source]#","").replace("\n","")
#                 func_text = func_soup.find("dd").find('p').text
#                 em = func_soup.find_all(attrs={"class":"field-odd"})
#                 param_names = em[-1].find_all("dt")
#                 desc_list = em[-1].find_all("dd")

#                 # if len(param_names)!=len(desc_list):
#                 #     print(func_name)
#                 #     print(f"param_names={len(param_names)} and desc_list={len(desc_list)}")
#                 curr_dict = {"function_name":func_name,"full_function":full_function,"function_text":func_text,"parameter_names_desc":[]}
#                 # first_level[parent]['function_definitions'].append()
#                 for pn,dn in zip(param_names,desc_list):
#                     param_name = pn.text
#                     if param_name == "**kwargs": continue
#                     param_desc = dn.text
#                     curr_dict['parameter_names_desc'].append({"param_name":param_name,"param_desc":param_desc})
#                 first_level[parent]['function_definitions'].append(curr_dict)
#             except Exception as e:
#                 print(e)
#                 print(func_name,func_text)
#     return first_level

# function_def_dict = get_param_data(first_level)

In [126]:
type(function_def_dict)

dict

In [127]:
import json
filename = "pandas_function.json"

# Serialize the dictionary to a JSON string
json_data = json.dumps(function_def_dict, ensure_ascii=False)

# Write the JSON string to a file with UTF-8 encoding
with open(filename, "w", encoding="utf-8") as json_file:
    json_file.write(json_data)

print("Data has been successfully written to", filename)

Data has been successfully written to pandas_function.json


In [128]:
filename = "pandas_function.json"

with open(filename, "r", encoding="utf-8") as json_file:
        # Load the JSON data from the file
        data = json.load(json_file)

In [85]:
func_url = 'https://pandas.pydata.org/docs/reference/api/pandas.read_clipboard.html'
func_url = 'https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.nanoseconds.html'
func_response = requests.get(func_url)
func_soup = BeautifulSoup(func_response.content, "lxml",from_encoding="utf-8")

In [36]:
func_name = func_soup.find("h1").text.replace("#","") #remove #

In [37]:
func_name

'pandas.Series.dt.nanoseconds'

In [38]:
elem = func_soup.find(attrs={"class":"sig sig-object py"})
full_function = elem.text.replace("[source]#","").replace("\n","")
func_text = func_soup.find("dd").find('p').text

In [39]:
func_text

'Number of nanoseconds (>= 0 and less than 1 microsecond) for each element.'

In [40]:
param_names = func_soup.find(attrs={"class":"field-list"})

In [134]:
em = param_names.find_all(attrs={"class":"field-odd"})

In [44]:
param_names

In [48]:
# len(em[-1].find_all('dt'))
em[-1].find_all('dt')

[<dt><strong>extra_args</strong><span class="classifier">list[str], default None</span></dt>,
 <dt><strong>run_doctests</strong><span class="classifier">bool, default False</span></dt>]

In [49]:
import re
def remove_non_english(sentence):
    # Define a regular expression pattern to match non-English characters
    non_english_pattern = re.compile(r'[^a-zA-Z0-9\s,.!?}(){]+\\')
    
    # Replace non-English characters with an empty string
    cleaned_sentence = non_english_pattern.sub('', sentence)
    # cleaned_sentence = re.sub('â','',sentence)
    # cleaned_sentence = re.sub('â','',sentence)
    return cleaned_sentence
for pname,param_desc in zip(em[-1].find_all('dt'),em[-1].find_all('dd')):
    # print('-'*100)
    param_name_text = pname.text
    desc_text = param_desc.text
    if "**kwargs" in param_name_text: continue
    process_param_name = remove_non_english(param_name_text)
    process_desc_text = remove_non_english(desc_text)
    # print(process_param_name)
    # print(process_desc_text)
    print(pname.text.strip())
    print(param_desc.text.strip())
    print('-'*100)

extra_argslist[str], default None
Extra marks to run the tests.
----------------------------------------------------------------------------------------------------
run_doctestsbool, default False
Whether to only run the Python and Cython doctests. If you would like to run
both doctests/regular tests, just append ââdoctest-modulesâ/ââdoctest-cythonâ
to extra_args.
----------------------------------------------------------------------------------------------------


In [19]:
func_soup.find(attrs={"class":"field-list"}).find(attrs={"class":"classifier"})

<span class="classifier">str, default â\s+â</span>

In [20]:
em = func_soup.find_all(attrs={"class":"field-odd"})

In [21]:
em[-1].find_all("dt")

[<dt><strong>sep</strong><span class="classifier">str, default â\s+â</span></dt>,
 <dt><strong>dtype_backend</strong><span class="classifier">{ânumpy_nullableâ, âpyarrowâ}, default ânumpy_nullableâ</span></dt>,
 <dt><strong>**kwargs</strong></dt>]

In [22]:
for p in em[-1].find_all("p"):

    if p.find(attrs={"class":"versionmodified added"}) is None and p.find(attrs={"class":"versionmodified changed"}) is None:
        print(p.text)
        print('-'*100)

A string or regex delimiter. The default of '\\s+' denotes
one or more whitespace characters.
----------------------------------------------------------------------------------------------------
Back-end data type applied to the resultant DataFrame
(still experimental). Behaviour is as follows:
----------------------------------------------------------------------------------------------------
"numpy_nullable": returns nullable-dtype-backed DataFrame
(default).
----------------------------------------------------------------------------------------------------
"pyarrow": returns pyarrow-backed nullable ArrowDtype
DataFrame.
----------------------------------------------------------------------------------------------------
See read_csv() for the full argument list.
----------------------------------------------------------------------------------------------------
